In [1]:
import numpy as np
import pandas as pd
import regex as re

import seaborn
from matplotlib import pyplot as plt

import duckdb, sqlalchemy

import array as arr

In [2]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [3]:
trees_df = pd.read_csv('Street_Tree_List-FILTERED.csv')

In [4]:
trees_df = trees_df.rename(columns = {'qSpecies' : 'scientific_name'})

trees_df.insert(5, 'common_name', (trees_df['scientific_name'].str.partition(' :: ')[2]).astype(str))
trees_df['scientific_name'] = (trees_df['scientific_name'].str.partition(' ::')[0]).astype(str)


trees_df.insert(3, 'genus', (trees_df['scientific_name'].str.partition(' ')[0]).astype(str))
trees_df.insert(4, 'species', (trees_df['scientific_name'].str.partition(' ')[2]).astype(str))


trees_df["PlantDate"] = pd.to_datetime(trees_df["PlantDate"])

trees_df.insert(10, 'Year', trees_df['PlantDate'].dt.year)
trees_df.insert(11, 'Month', trees_df['PlantDate'].dt.month_name())

trees_df


,TreeID,qLegalStatus,scientific_name,genus,species,qAddress,SiteOrder,common_name,qSiteInfo,qCaretaker,Year,Month,PlantDate,DBH,PlotSize,Latitude,Longitude
0,200311,DPW Maintained,Liquidambar styraciflua 'Rotundiloba',Liquidambar,styraciflua 'Rotundiloba',300 Spear St,9.0,Roundleaf sweetgum,Sidewalk: Curb side : Yard,Private,NaN,NaN,NaT,5,NaN,37.789193,-122.390006
1,10425,DPW Maintained,Schinus terebinthifolius,Schinus,terebinthifolius,1419 Haight St,2.0,Brazilian Pepper,Sidewalk: Curb side : Cutout,DPW,NaN,NaN,NaT,90,3x3,37.770094,-122.445693
2,43163,DPW Maintained,Acacia melanoxylon,Acacia,melanoxylon,222 Madison St,1.0,Blackwood Acacia,Sidewalk: Curb side : Cutout,Private,2000.0,July,2000-07-11,8,NaN,37.726036,-122.422690
3,188921,DPW Maintained,Callistemon citrinus,Callistemon,citrinus,2001 25th St,16.0,Lemon Bottlebrush,Sidewalk: Property side : Yard,Private,NaN,NaN,NaT,6,Width 8ft,37.751955,-122.399482
4,643,DPW Maintained,Ficus microcarpa,Ficus,microcarpa,3224 24th St,1.0,Chinese Banyan,Sidewalk: Curb side : Cutout,Private,NaN,NaN,NaT,18,3x3,37.752398,-122.416772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37113,141243,DPW Maintained,Magnolia grandiflora,Magnolia,grandiflora,31 COMMONWEALTH AVE,1.0,Southern Magnolia,Sidewalk: Curb side : Yard,Private,NaN,NaN,NaT,10,Width 0ft,37.785434,-122.455909
37114,203553,DPW Maintained,Agonis flexuosa,Agonis,flexuosa,259 Faxon Ave,1.0,Peppermint Willow,Sidewalk: Curb side : Cutout,Private,NaN,NaN,NaT,2,Width 3ft,37.719278,-122.460296
37115,61778,DPW Maintained,Eriobotrya deflexa,Eriobotrya,deflexa,2219 24th St,1.0,Bronze Loquat,Sidewalk: Curb side : Cutout,Private,1985.0,January,1985-01-22,13,NaN,37.753203,-122.401882
37116,190242,DPW Maintained,Ficus microcarpa nitida 'Green Gem',Ficus,microcarpa nitida 'Green Gem',60 New Montgomery St,3.0,Indian Laurel Fig Tree 'Green Gem',Sidewalk: Curb side : Pot,Private,NaN,NaN,NaT,2,Width 3ft,37.788459,-122.401854


In [5]:
trees_df.to_csv('trees.csv')

In [6]:
%sql genus_df << SELECT genus, COUNT(*) as count \
FROM trees_df GROUP BY genus, \
ORDER BY COUNT(*) ASC;

Returning data to local variable genus_df


In [7]:
genus_df.insert(2, 'percentage', (genus_df['count']/37118)*100)

In [8]:
genus_df.to_csv('genus.csv')

In [9]:
%sql species_df << SELECT scientific_name, common_name, genus, COUNT(scientific_name) as count \
FROM trees_df GROUP BY scientific_name, common_name, genus \
ORDER BY COUNT(scientific_name) DESC;

Returning data to local variable species_df


In [10]:
species_df.insert(3, 'percentage', (species_df['count']/37118)*100)

In [11]:
species_df

,scientific_name,common_name,genus,percentage,count
0,Platanus x hispanica,Sycamore: London Plane,Platanus,7.093593,2633
1,Metrosideros excelsa,New Zealand Xmas Tree,Metrosideros,5.218492,1937
2,Lophostemon confertus,Brisbane Box,Lophostemon,4.652729,1727
3,Pittosporum undulatum,Victorian Box,Pittosporum,4.496471,1669
4,Tristaniopsis laurina,Swamp Myrtle,Tristaniopsis,4.248613,1577
...,...,...,...,...,...
414,Juglans hindsii,Walnut: Black (n.calif),Juglans,0.002694,1
415,Metrosideros spp,,Metrosideros,0.002694,1
416,Acacia dealbata,Silver Wattle,Acacia,0.002694,1
417,Phoenix rupicola,Cliff date palm,Phoenix,0.002694,1


In [12]:
species_df.to_csv('species.csv')

In [13]:
common_trees = species_df[species_df['percentage'] >= 1]

In [14]:
common_trees = common_trees['scientific_name'].tolist()

print(common_trees)
print(len(common_trees))

['Platanus x hispanica', 'Metrosideros excelsa', 'Lophostemon confertus', 'Pittosporum undulatum', 'Tristaniopsis laurina', "Ficus microcarpa nitida 'Green Gem'", 'Magnolia grandiflora', 'Prunus cerasifera', "Arbutus 'Marina'", "Prunus serrulata 'Kwanzan'", 'Acacia melanoxylon', 'Olea europaea', 'Corymbia ficifolia', 'Maytenus boaria', 'Callistemon citrinus', 'Ginkgo biloba', 'Pyrus calleryana', 'Ulmus parvifolia', 'Ligustrum lucidum', 'Eriobotrya deflexa', 'Pinus radiata', 'Pyrus kawakamii', 'Cordyline australis']
23


In [15]:
most_common_trees_df = trees_df[trees_df["scientific_name"].isin(common_trees)]

most_common_trees_df

,TreeID,qLegalStatus,scientific_name,genus,species,qAddress,SiteOrder,common_name,qSiteInfo,qCaretaker,Year,Month,PlantDate,DBH,PlotSize,Latitude,Longitude
2,43163,DPW Maintained,Acacia melanoxylon,Acacia,melanoxylon,222 Madison St,1.0,Blackwood Acacia,Sidewalk: Curb side : Cutout,Private,2000.0,July,2000-07-11,8,NaN,37.726036,-122.422690
3,188921,DPW Maintained,Callistemon citrinus,Callistemon,citrinus,2001 25th St,16.0,Lemon Bottlebrush,Sidewalk: Property side : Yard,Private,NaN,NaN,NaT,6,Width 8ft,37.751955,-122.399482
6,133773,DPW Maintained,Platanus x hispanica,Platanus,x hispanica,2460 Scott St,1.0,Sycamore: London Plane,Sidewalk: Curb side : Cutout,Private,NaN,NaN,NaT,16,Width 4ft,37.792467,-122.439533
8,209974,DPW Maintained,Metrosideros excelsa,Metrosideros,excelsa,634 Miramar Ave,1.0,New Zealand Xmas Tree,Sidewalk: Curb side : Cutout,Private,NaN,NaN,NaT,6,Width 3ft,37.728313,-122.458106
9,9999,DPW Maintained,Ficus microcarpa nitida 'Green Gem',Ficus,microcarpa nitida 'Green Gem',126 Guerrero St,1.0,Indian Laurel Fig Tree 'Green Gem',Sidewalk: Curb side : Cutout,DPW,NaN,NaN,NaT,18,5x12,37.769212,-122.424656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37111,54849,DPW Maintained,Prunus cerasifera,Prunus,cerasifera,2076 Grove St,1.0,Cherry Plum,Sidewalk: Curb side : Cutout,Private,1989.0,April,1989-04-03,8,NaN,37.774373,-122.450701
37112,21647,DPW Maintained,Magnolia grandiflora,Magnolia,grandiflora,1660 Sutter St,2.0,Southern Magnolia,Sidewalk: Curb side : Cutout,Private,NaN,NaN,NaT,12,3X2,37.786868,-122.427688
37113,141243,DPW Maintained,Magnolia grandiflora,Magnolia,grandiflora,31 COMMONWEALTH AVE,1.0,Southern Magnolia,Sidewalk: Curb side : Yard,Private,NaN,NaN,NaT,10,Width 0ft,37.785434,-122.455909
37115,61778,DPW Maintained,Eriobotrya deflexa,Eriobotrya,deflexa,2219 24th St,1.0,Bronze Loquat,Sidewalk: Curb side : Cutout,Private,1985.0,January,1985-01-22,13,NaN,37.753203,-122.401882


In [16]:
most_common_trees_df.to_csv('commonfull.csv')

In [17]:
common_trees_df = species_df[species_df['percentage'] >= 2.1]

In [18]:
common_trees_df.to_csv('common.csv')

In [19]:
average_dbh_df = most_common_trees_df.groupby(['scientific_name', 'common_name'])['DBH'].mean()

average_dbh_df.reset_index(name='avg_dbh')
average_dbh_df = average_dbh_df.round(2)

In [20]:
most_planted_month_df = most_common_trees_df.groupby(['scientific_name', 'common_name'])['Month'].apply(lambda x: pd.Series.mode(x)[0])

most_planted_month_df.reset_index(name='month')

,scientific_name,common_name,month
0,Acacia melanoxylon,Blackwood Acacia,June
1,Arbutus 'Marina',Hybrid Strawberry Tree,March
2,Callistemon citrinus,Lemon Bottlebrush,July
3,Cordyline australis,Dracena Palm,March
4,Corymbia ficifolia,Red Flowering Gum,March
5,Eriobotrya deflexa,Bronze Loquat,February
6,Ficus microcarpa nitida 'Green Gem',Indian Laurel Fig Tree 'Green Gem',June
7,Ginkgo biloba,Maidenhair Tree,March
8,Ligustrum lucidum,Glossy Privet,September
9,Lophostemon confertus,Brisbane Box,February


In [21]:
caretakers_df = most_common_trees_df.groupby(['scientific_name', 'common_name'])['qCaretaker'].apply(pd.Series.mode)

caretakers_df.reset_index(name='common_caretaker')

,scientific_name,common_name,level_2,common_caretaker
0,Acacia melanoxylon,Blackwood Acacia,0,Private
1,Arbutus 'Marina',Hybrid Strawberry Tree,0,Private
2,Callistemon citrinus,Lemon Bottlebrush,0,Private
3,Cordyline australis,Dracena Palm,0,Private
4,Corymbia ficifolia,Red Flowering Gum,0,Private
5,Eriobotrya deflexa,Bronze Loquat,0,Private
6,Ficus microcarpa nitida 'Green Gem',Indian Laurel Fig Tree 'Green Gem',0,Private
7,Ginkgo biloba,Maidenhair Tree,0,Private
8,Ligustrum lucidum,Glossy Privet,0,Private
9,Lophostemon confertus,Brisbane Box,0,Private


In [22]:
details_df = pd.merge(average_dbh_df, most_planted_month_df, how="inner", on=['scientific_name', 'common_name'])

details_df

,,DBH,Month
scientific_name,common_name,,
Acacia melanoxylon,Blackwood Acacia,15.86,June
Arbutus 'Marina',Hybrid Strawberry Tree,5.82,March
Callistemon citrinus,Lemon Bottlebrush,9.01,July
Cordyline australis,Dracena Palm,9.30,March
Corymbia ficifolia,Red Flowering Gum,19.65,March
Eriobotrya deflexa,Bronze Loquat,6.14,February
Ficus microcarpa nitida 'Green Gem',Indian Laurel Fig Tree 'Green Gem',14.46,June
Ginkgo biloba,Maidenhair Tree,5.61,March
Ligustrum lucidum,Glossy Privet,9.87,September


In [23]:
details_df = pd.merge(details_df, caretakers_df, how="inner", on=['scientific_name', 'common_name'])
details_df = details_df.rename(columns = {'qCaretaker' : 'common_caretaker'})

details_df = details_df.head(n=12)

details_df

,,DBH,Month,common_caretaker
scientific_name,common_name,,,
Acacia melanoxylon,Blackwood Acacia,15.86,June,Private
Arbutus 'Marina',Hybrid Strawberry Tree,5.82,March,Private
Callistemon citrinus,Lemon Bottlebrush,9.01,July,Private
Cordyline australis,Dracena Palm,9.30,March,Private
Corymbia ficifolia,Red Flowering Gum,19.65,March,Private
Eriobotrya deflexa,Bronze Loquat,6.14,February,Private
Ficus microcarpa nitida 'Green Gem',Indian Laurel Fig Tree 'Green Gem',14.46,June,Private
Ginkgo biloba,Maidenhair Tree,5.61,March,Private
Ligustrum lucidum,Glossy Privet,9.87,September,Private


In [24]:
details_df.to_csv('species_averages.csv')

In [25]:
common_genus = genus_df[genus_df['percentage'] >= 1]
common_genus = common_genus['genus'].tolist()

print(common_genus)
print(len(common_genus))

['Cordyline', 'Cupressus', 'Melaleuca', 'Ligustrum', 'Eriobotrya', 'Acer', 'Eucalyptus', 'Pinus', 'Ginkgo', 'Maytenus', 'Ulmus', 'Corymbia', 'Callistemon', 'Olea', 'Acacia', 'Pyrus', 'Arbutus', 'Lophostemon', 'Magnolia', 'Tristaniopsis', 'Metrosideros', 'Pittosporum', 'Ficus', 'Platanus', 'Prunus']
25


In [26]:
common_genus_df = genus_df[genus_df['percentage'] >= 1]
common_genus_df = common_genus_df.rename(columns = {'genus' : 'name'})

common_genus_df.insert(2, 'Type', value="genus")
common_genus_df.insert(3, 'genus', value=common_genus_df['name'])

In [27]:
genus_species_df = species_df[species_df['genus'].isin(common_genus)]
genus_species_df.reset_index

<bound method DataFrame.reset_index of                   scientific_name                       common_name  \
0            Platanus x hispanica            Sycamore: London Plane   
1            Metrosideros excelsa             New Zealand Xmas Tree   
2           Lophostemon confertus                      Brisbane Box   
3           Pittosporum undulatum                     Victorian Box   
4           Tristaniopsis laurina                      Swamp Myrtle   
..                            ...                               ...   
410                Acer japonicum                    Japanese Maple   
411               Pinus torreyana                       Torrey Pine   
415              Metrosideros spp                                     
416               Acacia dealbata                     Silver Wattle   
418  Pyrus calleryana 'Cleveland'  Ornamental Pear Tree 'Cleveland'   

             genus  percentage  count  
0         Platanus    7.093593   2633  
1     Metrosideros    5.2184

In [28]:
# common_species_df = species_df[species_df['percentage'] >= 1]
common_species_df = genus_species_df.rename(columns = {'scientific_name' : 'name'})

common_species_df.insert(2, 'Type', value="species")

In [29]:
print(common_species_df)

                             name                       common_name     Type  \
0            Platanus x hispanica            Sycamore: London Plane  species   
1            Metrosideros excelsa             New Zealand Xmas Tree  species   
2           Lophostemon confertus                      Brisbane Box  species   
3           Pittosporum undulatum                     Victorian Box  species   
4           Tristaniopsis laurina                      Swamp Myrtle  species   
..                            ...                               ...      ...   
410                Acer japonicum                    Japanese Maple  species   
411               Pinus torreyana                       Torrey Pine  species   
415              Metrosideros spp                                    species   
416               Acacia dealbata                     Silver Wattle  species   
418  Pyrus calleryana 'Cleveland'  Ornamental Pear Tree 'Cleveland'  species   

             genus  percentage  count  

In [30]:
common_genus_df.to_csv('common_genus.csv')
common_species_df.to_csv('common_species.csv')

In [31]:
nodes_df = pd.concat([common_genus_df, common_species_df], keys=['genus', 'species'], ignore_index=True)
nodes_df = nodes_df.drop(nodes_df.index[[1, 2, 4, 6, 7, 8]])
nodes_df.reset_index(drop=True, inplace=True)

In [32]:
nodes_df.insert(0, 'id', value=nodes_df.index + 1)
# nodes_df = nodes_df.set_index('genus')

nodes_df

,id,name,count,Type,genus,percentage,common_name
0,1,Cordyline,373,genus,Cordyline,1.004903,NaN
1,2,Ligustrum,518,genus,Ligustrum,1.395549,NaN
2,3,Acer,532,genus,Acer,1.433267,NaN
3,4,Maytenus,766,genus,Maytenus,2.063689,NaN
4,5,Ulmus,800,genus,Ulmus,2.155289,NaN
...,...,...,...,...,...,...,...
188,189,Acer japonicum,1,species,Acer,0.002694,Japanese Maple
189,190,Pinus torreyana,1,species,Pinus,0.002694,Torrey Pine
190,191,Metrosideros spp,1,species,Metrosideros,0.002694,
191,192,Acacia dealbata,1,species,Acacia,0.002694,Silver Wattle


In [33]:
# def set_value(genus):
#     name = genus
#     df = nodes_df.set_index('name')
#     id = df.at[name, 'id']
#     return id


# nodes_df['group'] = nodes_df['genus'].apply(set_value)

# nodes_df

In [34]:
nodes_df

,id,name,count,Type,genus,percentage,common_name
0,1,Cordyline,373,genus,Cordyline,1.004903,NaN
1,2,Ligustrum,518,genus,Ligustrum,1.395549,NaN
2,3,Acer,532,genus,Acer,1.433267,NaN
3,4,Maytenus,766,genus,Maytenus,2.063689,NaN
4,5,Ulmus,800,genus,Ulmus,2.155289,NaN
...,...,...,...,...,...,...,...
188,189,Acer japonicum,1,species,Acer,0.002694,Japanese Maple
189,190,Pinus torreyana,1,species,Pinus,0.002694,Torrey Pine
190,191,Metrosideros spp,1,species,Metrosideros,0.002694,
191,192,Acacia dealbata,1,species,Acacia,0.002694,Silver Wattle


In [35]:
nodes_df.to_csv('bubbles.csv')

In [36]:
def getNodeID(name):
    df = nodes_df.set_index('name')
    id = df.at[name, 'id']
    return id

getNodeID('Prunus sp')

176

In [37]:
# links = []


# for i in range(len(nodes_df)):
#     element = nodes_df.loc[i]
#     if (element['Type'] == 'species'):
#         sourceGenus = element['genus']
        
#         sourceID = getNodeID(sourceGenus)
#         targetID = getNodeID(element['name'])
#         strength = 1
        
#         link = { 'source': sourceID, 'target': targetID, 'strength': strength };
#         links.append(link);
    
        
                            
# print(links)
# print(len(links))

# links = pd.DataFrame(links, columns = ['source', 'target'])

In [38]:
# links.to_csv('links.csv')

In [39]:
trees_genus_df = genus_df[genus_df['percentage'] >= 1]

trees_genus_df.reset_index(drop=True, inplace=True)


In [40]:
trees_genus_df.insert(0, 'id', value=trees_genus_df.index + 1)

In [41]:
# trees_genus_df[-1] = [17, "San Francisco's Trees", 5000, 100]
# trees_genus_df.index = df.index + 1  # shifting index
# trees_genus_df = trees_genus_df.sort_index() 

list_row = [26, "San Francisco's Trees", 2821, 100]
trees_genus_df.loc[len(trees_genus_df)] = list_row

/var/folders/5n/08myl3n11bn224bdxw_yfvjr0000gn/T/ipykernel_35095/1497002186.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trees_genus_df.loc[len(trees_genus_df)] = list_row


In [42]:
trees_genus_df

,id,genus,count,percentage
0,1,Cordyline,373,1.004903
1,2,Cupressus,388,1.045315
2,3,Melaleuca,453,1.220432
3,4,Ligustrum,518,1.395549
4,5,Eriobotrya,525,1.414408
5,6,Acer,532,1.433267
6,7,Eucalyptus,619,1.667655
7,8,Pinus,639,1.721537
8,9,Ginkgo,708,1.907430
9,10,Maytenus,766,2.063689


In [43]:
trees_genus_df.to_csv("genus_network.csv")

In [44]:
hopes = []


for i in range(len(trees_genus_df) - 1):
    element = trees_genus_df.loc[i]
       
#     sourceID = 17
    targetID = element['id']
        

    hope = { 'source': 26, 'target': targetID};
    hopes.append(hope);
    
        
                            
print(hopes)
print(len(hopes))

hopes_df = pd.DataFrame(hopes, columns = ['source', 'target'])

[{'source': 26, 'target': 1}, {'source': 26, 'target': 2}, {'source': 26, 'target': 3}, {'source': 26, 'target': 4}, {'source': 26, 'target': 5}, {'source': 26, 'target': 6}, {'source': 26, 'target': 7}, {'source': 26, 'target': 8}, {'source': 26, 'target': 9}, {'source': 26, 'target': 10}, {'source': 26, 'target': 11}, {'source': 26, 'target': 12}, {'source': 26, 'target': 13}, {'source': 26, 'target': 14}, {'source': 26, 'target': 15}, {'source': 26, 'target': 16}, {'source': 26, 'target': 17}, {'source': 26, 'target': 18}, {'source': 26, 'target': 19}, {'source': 26, 'target': 20}, {'source': 26, 'target': 21}, {'source': 26, 'target': 22}, {'source': 26, 'target': 23}, {'source': 26, 'target': 24}, {'source': 26, 'target': 25}]
25


In [45]:
hopes_df.to_csv('edges.csv')